In [1]:
import keras_tuner as kt
from tensorflow import keras

import sys 
sys.path.append("../")

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy


In [3]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
# CIFAR10 Dataset
from modules.utils import load_data
X_train, y_train,X_test,y_test = load_data(mode='all')
num_training = 49000
num_validation = 1000

X_val = X_train[-num_validation:, :]
y_val = y_train[-num_validation:]

X_train = X_train[:num_training, :]
y_train = y_train[:num_training]

# Preprocessing: subtract the mean value across every dimension for training data, and reshape it to be RGB size
mean_image = np.mean(X_train, axis=0)
X_train = X_train.astype(np.float32) - mean_image.astype(np.float32)
X_val = X_val.astype(np.float32) - mean_image
X_test = X_test.astype(np.float32) - mean_image

X_train = X_train.reshape(-1,3,32,32).transpose(0,2,3,1) / 255
X_val = X_val.reshape(-1,3,32,32).transpose(0,2,3,1) / 255
X_test = X_test.reshape(-1,3,32,32).transpose(0,2,3,1) / 255

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

y_train_dummy = tf.keras.utils.to_categorical(y_train)
y_val_dummy = tf.keras.utils.to_categorical(y_val)
y_test_dummy = tf.keras.utils.to_categorical(y_test)
print('Train labels shape (one-hot): ', y_train_dummy.shape)
print('Validation labels shape (one-hot): ', y_val_dummy.shape)
print('Test labels shape (one-hot): ', y_test_dummy.shape)

./data/cifar-10-python.tar.gz already exists. Begin extracting...
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)
Train labels shape (one-hot):  (49000, 10)
Validation labels shape (one-hot):  (1000, 10)
Test labels shape (one-hot):  (10000, 10)


In [11]:
from modules.CNN_withSpectralPooling import build_model

In [10]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [17]:
def build_hyperparamter_model(hp):
    input_shape=(32,32,3)
    layers_count = hp.Int("layers_count", min_value=2, max_value=8, step=1)
    gamma = hp.Float("gamma", min_value=0.75, max_value=.95)
    
    frequency_dropout_alpha = hp.Float("frequency_dropout_alpha", min_value=0.1, max_value=1)
    
    frequency_dropout_beta = hp.Float("frequency_dropout_beta", min_value=0.2, max_value=1)

    model = build_model(input_shape,M=layers_count, num_output=10,gamma=gamma,frequency_dropout_alpha=frequency_dropout_alpha,frequency_dropout_beta=frequency_dropout_beta)
    
    model.compile(optimizer= tf.keras.optimizers.Adam(),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

    
    return model

In [18]:
build_hyperparamter_model(kt.HyperParameters())


Building Conv layer 1 with Filters count 128 and cnn_kernel_size (3, 3)
Building Conv layer penultimate  with Filters count 160 and cnn_kernel_size (1, 1)
Building Conv layer penultimate  with Filters count 10 and cnn_kernel_size (1, 1)


In [19]:
tuner = kt.BayesianOptimization(
    hypermodel=build_hyperparamter_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=1,
    overwrite=True,
    directory="hyperparameter_search"
)

Building Conv layer 1 with Filters count 128 and cnn_kernel_size (3, 3)
Building Conv layer penultimate  with Filters count 160 and cnn_kernel_size (1, 1)
Building Conv layer penultimate  with Filters count 10 and cnn_kernel_size (1, 1)


In [20]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
layers_count (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 1, 'sampling': None}
gamma (Float)
{'default': 0.75, 'conditions': [], 'min_value': 0.75, 'max_value': 0.95, 'step': None, 'sampling': None}
frequency_dropout_alpha (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 1.0, 'step': None, 'sampling': None}
frequency_dropout_beta (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 1.0, 'step': None, 'sampling': None}


In [21]:
tuner.search(X_train, y_train, batch_size = 128, epochs=20, validation_data=(X_val, y_val))

Trial 5 Complete [00h 28m 29s]
val_accuracy: 0.7960000038146973

Best val_accuracy So Far: 0.8209999799728394
Total elapsed time: 02h 05m 17s
INFO:tensorflow:Oracle triggered exit


In [22]:
tuner.get_best_hyperparameters()[0]

In [23]:
best_model= tuner.get_best_models()[0]

Building Conv layer 1 with Filters count 128 and cnn_kernel_size (3, 3)
Building Conv layer 2 with Filters count 160 and cnn_kernel_size (3, 3)
Building Conv layer 3 with Filters count 192 and cnn_kernel_size (3, 3)
Building Conv layer 4 with Filters count 224 and cnn_kernel_size (3, 3)
Building Conv layer 5 with Filters count 256 and cnn_kernel_size (3, 3)
Building Conv layer 6 with Filters count 288 and cnn_kernel_size (3, 3)
Building Conv layer 7 with Filters count 320 and cnn_kernel_size (3, 3)
Building Conv layer penultimate  with Filters count 352 and cnn_kernel_size (1, 1)
Building Conv layer penultimate  with Filters count 10 and cnn_kernel_size (1, 1)
